In [3]:
from google.colab import files
uploaded = files.upload()

Saving 2023 서울특별시 광진구 주정차 단속 현황.csv to 2023 서울특별시 광진구 주정차 단속 현황.csv


In [2]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.graph_objects as go

import folium
from folium import plugins
import geopy
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

In [ ]:
# 미사용
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import plotly.graph_objects as go
from datetime import datetime
from plotnine import *
from pandas.tseries.offsets import YearEnd
from sklearn.linear_model import LinearRegression

# 한글 폰트 깨짐 현상 해결


In [9]:
# 실행 후 > 런타임 (런타임 다시 시작) > 다시 실행
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family = 'NanumBarunGothic')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

# csv 읽기

In [11]:
parking_crackdown_df = pd.read_csv('2023 서울특별시 광진구 주정차 단속 현황.csv', encoding = 'cp949')

parking_crackdown_df.head()

,단속일시,단속원금,단속동,단속장소,위반내용,견인지시
0,2023-01-01 0:25,32000,자양강변길,740,횡단보도,미견인
1,2023-01-01 3:36,32000,자양동,서울특별시 광진구 자양로 61,보도,미견인
2,2023-01-01 3:38,40000,자양동,서울특별시 광진구 자양로 61,보도,미견인
3,2023-01-01 7:10,40000,구의동,강변역 4번 출구,주차금지(황색점선)구역,미견인
4,2023-01-01 7:12,40000,구의동,지너스타워 주차장 앞,주정차금지(황색실선)구역,미견인


# 데이터 전처리

In [12]:
# 주소 데이터 전처리
def get_addrs(x):
    x1 = x.split(' ')
    return " ".join(x1[2:4])
    
# 주소 전처리
parking_crackdown_df['단속장소'] = parking_crackdown_df['단속장소'].apply(get_addrs)
parking_crackdown_df

,단속일시,단속원금,단속동,단속장소,위반내용,견인지시
0,2023-01-01 0:25,32000,자양강변길,,횡단보도,미견인
1,2023-01-01 3:36,32000,자양동,자양로 61,보도,미견인
2,2023-01-01 3:38,40000,자양동,자양로 61,보도,미견인
3,2023-01-01 7:10,40000,구의동,출구,주차금지(황색점선)구역,미견인
4,2023-01-01 7:12,40000,구의동,앞,주정차금지(황색실선)구역,미견인
...,...,...,...,...,...,...
4026,2023-02-28 20:43,32000,군자동,,주정차금지(황색실선)구역,미견인
4027,2023-02-28 21:23,8000,자양동,자양동 659-32,보도,미견인
4028,2023-02-28 21:38,32000,화양동,능동로 154,보도,미견인
4029,2023-02-28 22:36,32000,자양동,자양동 12-10,교차로,미견인


# 위도 경도 변환

In [ ]:
# 위도 반환 함수
def geocoding_lat(address):
    try:
        geo = geo_local.geocode(address)
        return geo.latitude
    except:
        return 0

# 경도 반환 함수
def geocoding_lon(address):
    try:
        geo = geo_local.geocode(address)
        return geo.longitude
    except:
        return 0

# 변환
parking_crackdown_df['위도'] = parking_crackdown_df['단속장소'].apply(geocoding_lat)
parking_crackdown_df['경도'] = parking_crackdown_df['단속장소'].apply(geocoding_lon)
parking_crackdown_df

In [ ]:
# 주소가 null값을 가지는 데이터 추출
nulls = parking_crackdown_df[parking_crackdown_df['위도'] == 0]
null_index = list(nulls.index)
null_index

parking_crackdown_df[parking_crackdown_df['위도'] == 0]

# 데이터 추출

In [ ]:
# csv저장
parking_crackdown_df.to_csv('2023 광진구 주정차단속 현황 데이터 (위도, 경도 추가).csv',encoding='cp949',index=False)



---



# 데이터 시각화

In [ ]:
def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding("대구 태전동")
print(crd['lat'])
print(crd['lng'])

35.9234127
128.5459483


In [ ]:
map = folium.Map(location=[37.5567054, 127.0917041], zoom_start=15)
map

# 예측 결과 출력